In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
df1 = pd.read_excel('D:\Darkweb Data\dnmarchives\silkroad2.tar\mastersr2.xlsx')

In [3]:
#df2 = pd.read_excel('')

In [4]:
df1.tail()

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
618861,"Twitter , Facebook , youtube , instagram services",profesorhouse,405.582,1.05,4.80,0.0,Belgium,Undeclared,Services,None,SilkRoad2,2014-09-30
618862,"500,000 High Retention Views | YouTube |",SideAlley,224.592,0.581441,3.65,0.0,Undeclared,Undeclared,Services,None,SilkRoad2,2014-09-30
618863,25 000 Real Twitter Followers!,profesorhouse,112.296,0.290721,4.80,0.0,Belgium,Undeclared,Services,None,SilkRoad2,2014-09-30
618864,"40,000 Facebook Followers Subscribers. >Cheap<",SideAlley,77.0031,0.199351,3.65,0.0,Undeclared,Undeclared,Services,None,SilkRoad2,2014-09-30
618865,2000 Youtube Real Likes!,profesorhouse,112.296,0.290721,4.80,0.0,Belgium,Undeclared,Services,None,SilkRoad2,2014-09-30


In [5]:
df1['Destination'].unique()

array(['Worldwide', 'Canada', 'United States',
       'Worldwide except Australia', 'European Union', 'Undeclared',
       'Australia', 'Sweden', 'Germany', 'United Kingdom',
       'United States &amp; Canada', nan, 'New Zealand', 'Norway',
       'Finland', 'Ireland', 'China', 'Switzerland', 'Denmark',
       'Azerbaijan', 'Netherlands', 'Italy', 'France',
       'European Union / UK / Worldwide', 'Worldwide\n ]',
       'United States & Canada', 'Argentina', 'Mexico', 'Japan', 'Tuvalu'],
      dtype=object)

In [6]:
region_dict = {
    'China': 'Asia',
    'Hong Kong, (China)': 'Asia',
    'Undeclared': 'Unknown',
    'India': 'Asia',
    'Canada': 'North America',
    'United States': 'North America',
    'Netherlands': 'EU',
    'United Kingdom': 'EU',
    'Germany': 'EU',
    'Belgium': 'EU',
    'South Africa': 'Africa',
    'Australia': 'Oceania',
    'Spain': 'EU',
    'Czech Republic': 'EU',
    'Sweden': 'EU',
    'Finland': 'EU',
    'New Zealand': 'Oceania',
    'Norway': 'Non-EU Europe',
    'Poland': 'EU',
    'Austria': 'EU',
    'Switzerland': 'Non-EU Europe',
    'Denmark': 'EU',
    'Ireland': 'EU',
    'Italy': 'EU',
    'Bulgaria': 'EU',
    'Slovenia': 'EU',
    'Armenia': 'Non-EU Europe',
    'Slovakia': 'EU',
    'Latvia': 'EU',
    'France': 'EU',
    'Hungary': 'EU',
    'Singapore': 'Asia',
    'Germany\n]': 'EU',
    'Colombia': 'Latin America',
    'Malaysia': 'Asia',
    'Israel': 'Middle East',
    'Japan': 'Asia',
    'Vatican (Holy See)': 'EU',
    'Angola': 'Africa',
    'Greece': 'EU',
    'Paraguay': 'Latin America',
    'Albania': 'Non-EU Europe',
    'Panama': 'Latin America',
    'Luxembourg': 'EU',
    'Kosovo': 'Non-EU Europe',
    'Mexico': 'North America',
    'Monaco': 'EU',
    'Argentina': 'Latin America',
    'Bolivia': 'Latin America',
    'Ukraine': 'Non-EU Europe',
    'Croatia': 'EU',
    'Denmark / UK (Top #4 Seller)': 'EU',
    'Lithuania': 'EU',
    'Romania': 'EU',
    'Reunion (FR)': 'Africa',
    'Saint Martin (FR)': 'Latin America',
    'Moldova, Republic of': 'Non-EU Europe',
    'Central America': 'Latin America',
    'Tuvalu': 'Oceania',
    'Thailand': 'Asia',
    'Afghanistan': 'Asia'
}

country_dict = {
    'China': 'China',
    'Hong Kong, (China)': 'China',
    'Undeclared': 'Unknown',
    'India': 'India',
    'Canada': 'Canada',
    'United States': 'United States',
    'Netherlands': 'Netherlands',
    'United Kingdom': 'United Kingdom',
    'Germany': 'Germany',
    'Belgium': 'Belgium',
    'South Africa': 'South Africa',
    'Australia': 'Australia',
    'Spain': 'Spain',
    'Czech Republic': 'Czech Republic',
    'Sweden': 'Sweden',
    'Finland': 'Finland',
    'New Zealand': 'New Zealand',
    'Norway': 'Norway',
    'Poland': 'Poland',
    'Austria': 'Austria',
    'Switzerland': 'Switzerland',
    'Denmark': 'Denmark',
    'Ireland': 'Ireland',
    'Italy': 'Italy',
    'Bulgaria': 'Bulgaria',
    'Slovenia': 'Slovenia',
    'Armenia': 'Armenia',
    'Slovakia': 'Slovakia',
    'Latvia': 'Latvia',
    'France': 'France',
    'Hungary': 'Hungary',
    'Singapore': 'Singapore',
    'Germany\n]': 'Germany',
    'Colombia': 'Colombia',
    'Malaysia': 'Malaysia',
    'Israel': 'Israel',
    'Japan': 'Japan',
    'Vatican (Holy See)': 'Holy See',
    'Angola': 'Angola',
    'Greece': 'Greece',
    'Paraguay': 'Paraguay',
    'Albania': 'Albania',
    'Panama': 'Panama',
    'Luxembourg': 'Luxembourg',
    'Kosovo': 'Kosovo',
    'Mexico': 'Mexico',
    'Monaco': 'Monaco',
    'Argentina': 'Argentina',
    'Bolivia': 'Bolivia',
    'Ukraine': 'Ukraine',
    'Croatia': 'Croatia',
    'Denmark / UK (Top #4 Seller)': 'United Kingdom',
    'Lithuania': 'Lithuania',
    'Romania': 'Romania',
    'Reunion (FR)': 'Reunion',
    'Saint Martin (FR)': 'Saint Martin',
    'Moldova, Republic of': 'Moldova',
    'Central America': 'Central America',
    'Tuvalu': 'Tuvalu',
    'Thailand': 'Thailand',
    'Afghanistan': 'Afghanistan',
}

df1['Origin'] = df1['Origin'].map(country_dict)
df1['Origin_region'] = df1['Origin'].map(region_dict)
#df1['Reviews'] = df1['Reviews'].astype(int)
df1.dtypes

Title             object
Sellerid          object
PriceUSD          object
PriceBTC          object
Rating           float64
Reviews          float64
Origin            object
Destination       object
Category          object
Subcategory       object
Market            object
Date              object
Origin_region     object
dtype: object

In [7]:
df1['Subcategory'].unique()

array(['None', 'Materials', 'Equipment', 'Orders', 'Goods',
       'Paraphernalia', nan, 'Clones', 'Concentrates', 'Cuttings',
       'Edibles', 'Hash', 'Pre', 'Seeds', 'Shake', 'Synthetic',
       'Topicals', 'Trim', 'Weed', 'Dioscorea', 'Ketamine', 'Mxe', 'Pcp',
       '5mapb', 'Butylone', 'Ethylone', 'Mdai', 'Mda', 'Mdma',
       'Methylone', 'Mpa', 'Pentedrone', 'Pills', 'Ah', 'Heroin',
       'Kratom', 'Opium', 'Barbiturates', 'Intoxicants', "Maoi's",
       'Nootropics', "Ssri's", 'Supplements', 'Tobacco', 'Analgesics',
       'Antidepressant', 'Generalhealth', 'Relaxants',
       'Sildenafilcitrate', 'Stimulants', '2cfamily', '4acofamily',
       '4hofamily', '5meofamily', 'Allad', 'Bufotenin', 'Dmt', 'Dox',
       'Dpt', 'Entheogens', 'Lsa', 'Lsd', 'Lsz', 'Met', 'Nbome', 'Salvia',
       'Shrooms', 'Anabolicsteroids', 'Antagonists',
       'Aromataseinhibitors', 'Clenbuterol', 'Drostanolone',
       'Fluoxymesterone', 'Hcg', 'Humangrowthhormones', 'Mesterelone',
       'Metabol